## 1. Solicitação de Cartões de Crédito
<p>Bancos comerciais recebem <em>muitas</em> solicitações de cartões de crédito. Muitos deles são rejeitados por vários motivos, como saldos altos de empréstimos, baixa renda ou muitas consultas em orgãos como SERARA e SPC sobre o relatório de crédito de um indivíduo, por exemplo. Analisar manualmente esses aplicativos é mundano, propenso a erros e demorado. Felizmente, essa tarefa pode ser automatizada com Machine Learning e praticamente todos os bancos comerciais fazem isso hoje em dia. Neste notebook, construiremos um preditor automático de aprovação de cartão de crédito usando técnicas ML, assim como os bancos reais fazem.</p>
<p><img src="https://assets.datacamp.com/production/project_558/img/credit_card.jpg" alt="Cartão de crédito em mãos"></p>
<p>Usaremos o <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">conjunto de dados de aprovação de cartão de crédito</a> do repositório de ML UCI. A estrutura deste notebook é a seguinte:</p>
<ul>
<li>Primeiro, começaremos carregando e visualizando o conjunto de dados.</li>
<li>Veremos que o conjunto de dados tem uma mistura de recursos numéricos e não numéricos, contendo valores de diferentes intervalos, além de conter várias entradas ausentes.</li>
<li>Teremos que pré-processar o conjunto de dados para garantir que o modelo de ML escolhido possa fazer boas previsões.</li>
<li>Depois que nossos dados estiverem em boa forma, faremos uma análise exploratória de dados para construir nossas intuições.</li>
<li>Por fim, criaremos um modelo possa prever se a solicitação de um cartão de crédito de um indivíduo será aceita.</li>
</ul>
<p>Primeiro, carregar e visualizar o conjunto de dados. Como são dados confidenciais, o colaborador do conjunto de dados tornou anônimos os nomes dos recursos.</p>

In [92]:
# Importando a biblioteca Pandas
import pandas as pd
# Carregando o dataset
cc_apps = pd.read_csv('datasets/cc_approvals.data', header=None)

# Inspecionando os dados
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


## 2. Inspecionando as solicitações
<p>A saída pode parecer um pouco confusa à primeira vista, mas vamos tentar descobrir os recursos mais importantes de um aplicativo de cartão de crédito. Os recursos deste conjunto de dados foram anonimizados para proteger a privacidade, mas <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html">este blog</a> nos dá uma bela boa visão geral das características prováveis. As características prováveis em um aplicativo de cartão de crédito típico são <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer </code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> e, finalmente, o <code> Approval Status</code>. Isso nos dá um bom ponto inicial e podemos mapear esses recursos em relação às colunas na saída. </p>
<p>Como podemos ver à primeira vista dos dados, o conjunto de dados tem uma mistura de recursos numéricos e não numéricos. Isso pode ser corrigido com algum pré-processamento, mas antes de fazermos isso, vamos analisar um pouco mais o dataset para ver se há outros problemas  que precisam ser corrigidos.</p>

In [94]:
# Imprimindo sumário de estatísticas
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

# Imprimindo informações relevantes sobre o DataFrame
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print("\n")

# Inspecionando 'missing values' no dataset

cc_apps.tail(17)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


## 3. Manipulando 'missing values' (1ª parte)
<p>Descobrimos alguns problemas que afetarão o desempenho de nossos modelos de aprendizado de máquina se não forem alterados:</p>
<ul>
<li>Nosso conjunto de dados contém dados numéricos e não numéricos (especificamente dados dos tipos <code>float64</code>, <code>int64</code> e <code>object</code>). Especificamente, os recursos 2, 7, 10 e 14 contêm valores numéricos (dos tipos float64, float64, int64 e int64, respectivamente) e todos os outros recursos contêm valores não numéricos.</li>
<li>O conjunto de dados também contém valores de vários intervalos. Alguns recursos têm um intervalo de valores de 0 a 28, alguns têm um intervalo de 2 a 67 e alguns têm um intervalo de 1.017 a 100.000. Além disso, podemos obter informações estatísticas úteis (como <code>mean</code> , <code>max</code> e <code>min</code>) sobre os recursos que possuem valores numéricos. </li>
<li>Finalmente, o conjunto de dados possui 'missing values' dos quais cuidaremos nesta tarefa. Os 'missing values no conjunto de dados são rotulados com '?', que pode ser visto na saída da última célula.</li>
</ul>
<p>Agora, vamos substituir temporariamente esses pontos de interrogação por 'NaNs'.</p>

In [96]:
# Importando a biblioteca numpy
import numpy as np
# Inspecionando 'missing values' no dataset
cc_apps.tail(17)

# Substituindo os '?'s por 'NaN'
cc_apps = cc_apps.replace('?', np.NaN)

# Inspecionando novamente os 'missing values'
cc_apps.tail(17)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


## 4.  Manipulando 'missing values' (2ª parte)
<p>Substituímos todos '?s' por 'NaNs'. Isso vai nos ajudar no próximo tratamento de 'missing value' que vamos realizar.</p>
<p>Uma questão importante que é levantada aqui é <em>por que estamos dando tanta importância aos valores ausentes</em>? Eles não podem ser simplesmente ignorados? Ignorar valores ausentes pode afetar fortemente o desempenho de um modelo de ML. Ao ignorar os valores ausentes, nosso modelo de aprendizado de máquina pode perder informações sobre o conjunto de dados que podem ser úteis para seu treinamento. Então, existem muitos modelos que não podem manipular 'missing values' implicitamente. </p>
<p>Então, para evitar esse problema, vamos imputar os 'missing values' com uma estratégia chamada imputação média.</p>

In [98]:
# Imputando os 'missing values' com 'mean imputation'
cc_apps.fillna(cc_apps.mean(), inplace=True)

# Contando o número de "NaNs' no dataset para verificação
cc_apps.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

## 5. Manipulando 'missing values' (3ª parte)
<p>Solucionamos com sucesso a questão dos 'missing values' presentes nas colunas numéricas. Ainda faltam alguns valores a serem imputados para as colunas 0, 1, 3, 4, 5, 6 e 13. Todas essas colunas contêm dados não numéricos e por isso a estratégia de 'mean imputation' não funcionaria aqui. É necessária uma abordagem diferente. </p>
<p>Vamos imputar esses 'missing values' com os valores mais frequentes conforme apresentados nas respectivas colunas. Esta é uma <a href="https://www.datacamp.com/community/tutorials/categorical-data">boa prática</a> quando se trata de imputar valores ausentes para dados categóricos em geral.</p>

In [100]:
# Iterando sobre cada coluna do 'cc_apps'
for col in cc_apps.columns:
    # Checando se a coluna é do tipo 'objeto'
    if cc_apps[col].dtypes == 'object':
        # Imputando com o valor mais frequente
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])

# Contando o número de "NaNs' no dataset e imprimindo as contagens para verificação

print(cc_apps.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


## 6. Preprocessamento dos dados (1ª parte)
<p>Os 'missing values' foram tratados com sucesso.</p>
<p>Ainda é necessário um pré-processamento de dados menor, mas ainda assim essencial, antes de prosseguirmos com a construção de Machine Learning. Vamos dividir essas etapas restantes de pré-processamento em três tarefas principais:</p>
<ol>
<li>Conversão dos dados não numéricos em numéricos.</li>
<li>Divisão dos dados em conjuntos de treinamento e teste. </li>
<li>Dimensionamento (scaling) dos valores das 'features' para um intervalo uniforme.</li>
</ol>
<p>Primeiro, converteremos todos os valores não numéricos em numéricos. Fazemos isso porque não apenas resulta em uma computação mais rápida, mas também muitos modelos de ML (XGBoost, por exemplo) e especialmente os desenvolvidos usando scikit-learn exigem que os dados estejam em um formato estritamente numérico. Faremos isso usando uma técnica chamada <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html">'label encoding'</a>.</p>

In [102]:
# Importando o LabelEncoder
from sklearn.preprocessing import LabelEncoder
# Instanciando LabelEncoder
le = LabelEncoder()
# Iterando sobre todos os valores de cada coluna e extraindo seus 'dtypes'
for col in cc_apps.columns.to_numpy():
    # Verificando se o dtype é um objeto
    if cc_apps[col].dtypes =='object':
    # Usando o LabelEncoder para realizar a transformação númerica
        cc_apps[col]=le.fit_transform(cc_apps[col])

## 7. Dividindo o dataset em 'train' e 'test' sets
<p>Convertemos com sucesso todos os valores não numéricos em numéricos.</p>
<p>Agora, dividiremos nossos dados em um 'train set' e um 'test set' para preparar nossos dados para duas fases diferentes da modelagem de aprendizado de máquina: treinamento e teste. Idealmente, nenhuma informação dos dados de teste deve ser usada para dimensionar os dados de treinamento ou deve ser usada para direcionar o processo de treinamento de um modelo de aprendizado de máquina. Portanto, primeiro dividimos os dados e, em seguida, aplicamos o dimensionamento.</p>
<p>Além disso, 'features' como <code>DriversLicense</code> e <code>ZipCode</code> não são tão importantes quanto as outras do dataset para prever aprovações de cartão de crédito. Devemos retirá-los para projetar nosso modelo de aprendizado de máquina com o melhor conjunto de recursos. Chamamos isso, em Data Science, de <em>'feature selection'</em>. </p>

In [104]:
# Importando train_test_split
from sklearn.model_selection import train_test_split
# Retirando as 'features' 11 e 13 e convertendo o DataFrame para uma 'NumPy array'
cc_apps = cc_apps.drop([11, 13], axis=1)
cc_apps = cc_apps.to_numpy()

# Separando 'features' e 'labels' em variáveis separadas
X,y = cc_apps[:,0:13] , cc_apps[:,13]

# Dividindo em 'train' e 'test' sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                y,
                                test_size=0.33,
                                random_state=42)

## 8. Preprocessamento dos dados (2ª parte)
<p>Os dados agora são divididos em dois conjuntos separados - treinamento e teste, respectivamente. Só nos resta uma etapa final de pré-processamento de dimensionamento antes de podermos ajustar um modelo de aprendizado de máquina aos dados. </p>
<p>Agora, vamos tentar entender o que esses valores escalonados significam no mundo real. Vamos usar <code>CreditScore</code> como exemplo. A pontuação de crédito de uma pessoa é a sua credibilidade com base em seu histórico de crédito. Quanto maior esse número, mais financeiramente confiável uma pessoa é considerada. Portanto, um <code>CreditScore</code> de 1 é o mais alto, pois estamos redimensionando todos os valores para o intervalo de 0 a 1.</p>

In [106]:
# Importando o MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
# Instanciando MinMaxScaler e usando-o para redimensionar X_train e X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

## 9. Ajustando o modelo de regressão logística ao train set
<p>Essencialmente, prever se uma solicitação de cartão de crédito será aprovada ou não é uma tarefa de <a href="https://en.wikipedia.org/wiki/Statistical_classification">classificação</a>. <a href="http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.names">De acordo com UCI</a>, nosso conjunto de dados contém mais instâncias que correspondem a Status "Negado" do que instâncias correspondentes ao status "Aprovado". Especificamente, das 690 instâncias, há 383 (55,5%) solicitações negadas e 307 (44,5%) solicitações aprovadas. </p>
<p>Isso nos dá uma referência. Um bom modelo de aprendizado de máquina deve ser capaz de prever com precisão o status das solicitações em relação a essas estatísticas.</p>
<p>Qual modelo devemos escolher? Uma pergunta a ser feita é: <em>as 'features' que afetam o processo de decisão de aprovação do cartão de crédito estão correlacionadas entre si?</em> Embora possamos medir a correlação, isso está fora do escopo deste projeto. Nossa intuição é de que eles de fato estão correlacionados por enquanto. Por causa dessa correlação, aproveitaremos o fato de que os modelos lineares generalizados têm um bom desempenho nesses casos. Vamos começar nossa modelagem de aprendizado de máquina com um modelo de regressão logística (um modelo linear generalizado).</p>

In [108]:
# Importando LogisticRegression
from sklearn.linear_model import LogisticRegression
# Instanciando o LogisticRegression classifier com valores 'default' para os parâmetros
logreg = LogisticRegression()

# Ajustando o 'logreg' para o 'train set'
logreg.fit(rescaledX_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## 10. Fazendo previsões e analisando desempenho
<p>Mas quão bem nosso modelo funciona?</p>
<p>Agora avaliaremos nosso modelo no 'test set' em relação à <a href="https://developers.google.com/machine-learning/crash-course/classification/accuracy">precisão da classificação</a> . Mas também veremos a <a href="http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">'confusion matrix'</a> do modelo. No caso de prever pedidos de cartão de crédito, é igualmente importante ver se nosso modelo de aprendizado de máquina é capaz de prever o status de aprovação dos pedidos como 'negados' que foram negados originalmente. Se nosso modelo não estiver tendo um bom desempenho nesse aspecto, ele pode acabar aprovando a solicitação que deveria ter sido aprovada. 'Confusion Matrix' nos ajuda a visualizar o desempenho do nosso modelo a partir desses aspectos. </p>

In [110]:
# Importando confusion_matrix
from sklearn.metrics import confusion_matrix
# Usando 'logreg' para prever instâncias do 'test set' e armazená-las
y_pred = logreg.predict(rescaledX_test)

# Obtendo o 'accuracy score' do modelo 'logreg' e imprimindo-o
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test, y_test))

# Imprimindo a 'confusion matrix' do modelo 'logreg'
print(confusion_matrix(y_test, y_pred))

Accuracy of logistic regression classifier:  0.8377192982456141
[[92 11]
 [26 99]]


## 11. Grid searching e melhorando o desempenho do modelo
<p>Nosso modelo gerou um resultado muito bom. Foi capaz de gerar uma pontuação de precisão de quase 84%.</p>
<p>Para a 'matrix confusion', o primeiro elemento da primeira linha da matriz  denota os verdadeiros negativos, significando o número de instâncias negativas (solicitações negadas) previstas corretamente pelo modelo. E o último elemento da segunda linha denota os verdadeiros positivos, ou seja, o número de instâncias positivas (solicitações aprovadas) previstos corretamente pelo modelo.</p>
<p>Vamos ver se podemos fazer melhor. Podemos realizar uma <a href="https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/">'Grid Search'</a> dos parâmetros do modelo para melhorar a capacidade do modelo em prever aprovações de cartão de crédito.</p>
A <p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">implementação de regressão logística do scikit-learn</a> consiste em diferentes hiperparâmetros, mas iremos utilizar o 'Grid Search' nos dois seguintes:</p>
<ul>
<li>tol</li>
<li>max_iter</li>
</ul>

In [112]:
# Importando GridSearchCV
from sklearn.model_selection import GridSearchCV
# Definindo o grid de valores para 'tol' e 'max_iter'
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

# Criando um dicionário onde 'tol' e 'max_iter' são 'keys' e as listas de seus valores são os 'values'
param_grid = dict(tol=tol, max_iter=max_iter)

## 12. Encontrando o modelo de melhor performance
<p>Nós definimos a grade de hiperparâmetros e os convertemos em um único formato de dicionário no qual <code>'GridSearchCV()'</code> espera como um de seus parâmetros. Agora, iniciaremos o 'Grid Search' para ver quais valores apresentam melhor desempenho.</p>
<p>Instanciaremos <code>'GridSearchCV()'</code> com nosso modelo <code>'logreg'</code> anterior com todos os dados que temos. Em vez de passar os conjuntos de 'train' e 'test' separadamente, forneceremos <code>'X'</code> (versão em escala) e <code>'y'</code>. Também instruiremos <code>'GridSearchCV()'</code> a realizar uma <a href="https://www.dataschool.io/machine-learning-with-scikit-learn/">'Cross Validation'</a > de cinco 'folds'.</p>
<p>Encerraremos o notebook armazenando a melhor pontuação alcançada e os respectivos melhores parâmetros.</p>
<p>Ao criar esse preditor de cartão de crédito, abordamos algumas das etapas de pré-processamento mais conhecidas, como <strong>'scaling'</strong>, <strong>'label encoding'</strong> e <strong>'missing value imputation</strong>. Terminamos com um pouco de <strong>Machine Learning</strong> para prever se a solicitação de um cartão de crédito de uma pessoa seria aprovada ou não de acordo com algumas informações sobre essa pessoa.</p>

In [114]:
# Instanciando 'GridSearchCV' com os parâmetros necessários
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Utilizando 'scaler' para redimensionar 'X' e atribuí-lo para 'rescaledX'
rescaledX = scaler.fit_transform(X)

# Ajustando os dados para 'grid_model'
grid_model_result = grid_model.fit(rescaledX, y)

# Sumarizando resultados
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.853623 using {'max_iter': 100, 'tol': 0.01}
